## Importar todos los modulos necesarios

In [328]:
import pandas as pd
import numpy as np
import uuid
from scipy.stats import truncnorm 

## Random Values Generators

In [332]:
# Funcion normal truncada
def generate_truncated_normal_data(min_val, max_val, std, mean, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    data = np.round(data, 3)
    return data

# Generar una serie de valores aleatorios de tipo categoria con tamanio relativo dependiente del parametro size
def generate_category_random_values(arr, size):
    return pd.Series(np.random.choice(arr, size=size))

# Generar un arreglo de valores aleatorios de tipo unique con tamanio relativo dependiente del parametro size
def generate_unique_random_values(size):
    return [uuid.uuid4().hex[:16] for _ in range(size)]

# Generar un arreglo de valores aleatorios de tipo date con tamanio relativo dependiente del parametro size
def generate_date_random_values(min, max, size):
    date_range = pd.date_range(start=min, end=max, freq='D')
    return np.random.choice(date_range, size)

# Generar un arreglo  de valores aleatorios de tipo foreign con tamanio relativo dependiente del parametro size
def generate_foreign_random_values(serie, size):
    return np.random.choice(serie.drop_duplicates(), size)

# Generar un arreglo  de valores aleatorios de tipo numeric con tamanio relativo dependiente del parametro size
def generate_numeric_random_values(min, max, std, mean, size):
    if std is not None and mean is not None:
        values = generate_truncated_normal_data(min, max, std, mean, size)
    else:
        values = np.random.uniform(min, max, size)
        values = np.round(values, 3)
    return values 

## Random Dataset Simulations Generators

In [505]:
# Generar n cantidad de registros categoricos de manera aleatoria
def get_categorical_dataset_simulated(simulation_extended, category_cols, size):
    # la columna n = 1 servira para realizar la funcion de agrupacion count en el dataframe
    simulation_extended['n'] = 1

    # Generamos la agrupacion por columnas categoricas con su respectiva funcion de agregacion count para obtener la frecuencia con la que aparecen cada agrupacion
    grouped_combination = simulation_extended.groupby(
        category_cols,
        as_index=False
    ).agg({
        'n': 'count'
    })

    # Renombramos los nombres de las columnas, en este caso seran las mismas columnas categoricas + la ultima columna n que era la relacionada con la frecuencia o conteo
    grouped_combination.columns = category_cols + ['count']

    # Agregamos otra columna extra de prob para almacenar la probabilidad con la que aparecen cada grupo de columnas categoricas
    grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

    # Generamos aleatoriamente un arreglo de indices en base a las combinaciones generadas anteriormente manteniendo sus probabilidades
    rand_index = np.random.choice(
        grouped_combination.index,
        size = size,
        p = list(grouped_combination['prob'])
    )

    # Retornamos los 100k registros aleatorios ya con sus respectivos valores, no como indices nada mas
    return grouped_combination.loc[rand_index, category_cols]   

# Generar los registros de manera aleatoria para las columnas numericas
def get_numeric_column_simulated( simulated , simulation_extended , category_cols , column_name ):
    # Funcion de agregacion para obtener el min, max, mean y std por cada agrupacion de columnas categoricas
    a1 = simulation_extended.groupby(
        category_cols
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )

    a1.columns = [c for c in category_cols] + ["Min", "Max", "Mean", "Std"]  # Renombra las columnas de a1

    ColumnSimulated = pd.DataFrame() # Inicializa un nuevo dataframe vacio en este caso
    
    # Recorre los indices de a1
    for i in a1.index:
        # Toma el registro completo del indice i, el registro completo son toda la fila de valores de ese indice
        rs = a1.loc[i]

        # reasigna final_simulation y crea una copia de todos los registros de simulated (que al principio era una copia de final_simulation sin la columna n) que contengan el indice i
        final_simulation = simulated.loc[i].copy()    

        # Genera los valores aleatorios en base a su resumen estadistico de min, max, mean y std
        data = generate_truncated_normal_data(
            rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"],
            rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"],
            1 if rs["Std"] == 0 else rs["Std"],
            rs["Mean"], 
            final_simulation.shape[0]
        )

        # Agregamos la nueva columna a final_simulation
        final_simulation[column_name] = data

        # Concatenamos el conjunto de registros anteriores con el registro actual recorrido del ciclo
        ColumnSimulated = pd.concat( [ ColumnSimulated , final_simulation  ] )

    return ColumnSimulated.reset_index(drop=True) # Retornamos el resultado final restableciendo los indices
    

## Helpers

In [508]:
# Funcion encargada de calcular la columna con mayor cantidad de registros (Esto en caso de que el valor random sea False)
def calc_greater(cols_arr):
    max_values = 0 # Inicia en cero suponiendo que no hay nada todavia al inicio
    
    for col in cols_arr:
        col_type = col['type']

        if col_type == 'foreign':
            pass
        
        if col_type == 'date':
            max_values = max(max_values, pd.date_range(start=col['values']['min'], end=col['values']['max']).size)

        if col_type == 'category':
            max_values = max(max_values, len(col['values']))

    return max_values

# Funcion encargada de evaluar que en efecto las columnas relacionadas existan asi como sus posibles columnas
def validate_relationships(all_dfs, reference_df):
    # Primero valido que exista el dataframe
    df_name, df_column = reference_df.split('.')

    # Validar por nombre del dataframe
    if df_name not in all_dfs:
        raise ValueError(f'El dataframe {df_name} no existe :(')

    # Sacamos las columnas validas
    columns = list(all_dfs[df_name].columns)

    # Validamos por columna
    if df_column not in columns:
        raise ValueError(f'La columna {df_column} no existe :( en el dataframe {df_name}. Las columnas validas son: {columns}.')

# Funcion que retorna un arreglo con todas las dependencias de un objeto que contiene alguna configuracion
def extract_dependencies(obj):
    dependencies = []
    
    for col in obj['columns']:
        if col['type'] == 'foreign':
            df_name = col['values'].split('.')[0]
            dependencies.append(df_name)
    return dependencies

# Funcion encargada de reordenar por dependencias el arreglo conf_list que puede que su orden no sea el de acuerdo a sus dependencias
def reorder_by_dependencies(arr): # Usando el algoritmo de insertion sort
    for i, value in enumerate(arr[1:], start=1):
        value_dependencies = extract_dependencies(value)
        j = i - 1

        # Validar que j no sea menor que cero y que las dependencias del valor actual recorrido esten en el valor de la izq
        while j >= 0 and any(dep in [d['ds'] for d in arr[:j+1]] for dep in value_dependencies):
            arr[j + 1] = arr[j]
            j -= 1
        
        # Remplazar el valor actual despues del elemento del que dependa
        arr[j + 1] = value
    return list(reversed(arr))

all_dfs = {} # Diccionario que contiene todos los dataframes creados

# Funcion que genera un dataframe a partir de su configuracion
def generate_df(sett):
    result_df = pd.DataFrame()

    # Aqui mandamos a llamar a la funcion calc_greater para determinar cual columna contiene la mayor cantidad de registros
    max_values = calc_greater(sett['columns']) if sett['random'] == False else sett['random_rows']
    
    for col in sett['columns']:
        col_name, col_type = col['name'], col['type']
        
        if col_type == 'category':
            values = col['values']

            if (len(values) != max_values):
                result_df[col_name] = generate_category_random_values(values, max_values)
            else:    
                result_df[col_name] = values

        if col_type == 'date':
            min = col['values']['min']
            max = col['values']['max']
            date_range = pd.date_range(start=min, end=max)
            
            if (date_range.size != max_values):
                result_df[col_name] = generate_date_random_values(min, max, max_values)
            else:    
                result_df[col_name] = pd.Series(date_range)

        if col_type == 'unique':
            result_df[col_name] = generate_unique_random_values(max_values)

        if col_type == 'foreign':
            col_reference_name, col_reference_area = col['values'].split('.')
            
            validate_relationships(all_dfs, col['values']) # Si no se traba aqui por un error de validacion entonces continua su ejecucion

            result_df[col_name] = generate_foreign_random_values(all_dfs[col_reference_name][col_reference_area], max_values)

        if col_type == 'numeric':
            if 'std' in col['values'].keys() and 'mean' in col['values'].keys():
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], col['values']['std'], col['values']['mean'], max_values)
            else:
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], None, None, max_values)
            
    result_df = pd.DataFrame(result_df)
    result_df.name = sett['ds']

    all_dfs[result_df.name] = result_df
    return result_df

## Main

In [511]:
def build_dataframes(conf_list):
    temp_conf_list = reorder_by_dependencies(conf_list.copy())
    df_arr = []

    for df in temp_conf_list:
        df_arr.append(generate_df(df))

    # Reordenar considerando el orden original (conf_list)
    return  [j for i in conf_list for j in df_arr if i['ds'] == j.name]

## Settings

In [556]:
d1 = {
    "ds": "dataset",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "Fecha",
            "type": "date",
            "values": {
                "min": "2024-01-01",
                "max": "2024-02-28"
            }
        }
    ],
    "random": False
}

d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}

d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        }, 
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 10000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 50000,
                "std": 20000,
                "mean": 25000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TIC", "FIN", "HR", "MKT"]
        },
        {
            "name": "Fecha",
            "type": "date",
            "values": {
                "min": "2024-01-01",
                "max": "2024-02-28"
            }
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset3.id"
        },
        {
            "name": "id",
            "type": "unique"
        }
    ],
    "random": False
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        }, 
        {
            "name": "area",
            "type": "category",
            "values": ["TIC", "FIN", "HR", "MKT"]
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 10000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 50000,
                "std": 20000,
                "mean": 25000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

d6 = {
    "ds": "dataset6",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "department",
            "type": "category",
            "values": ["Sales", "Engineering", "Support", "Administration"]
        },
        {
            "name": "location",
            "type": "category",
            "values": ["New York", "San Francisco", "Austin", "Seattle"]
        },
        {
            "name": "status",
            "type": "category",
            "values": ["Full-time", "Part-time", "Contract", "Intern"]
        },
        {
            "name": "salary",
            "type": "numeric",
            "values": {
                "min": 30000,
                "max": 120000
            }
        },
        {
            "name": "experience_years",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 40
            }
        },
        {
            "name": "bonus",
            "type": "numeric",
            "values": {
                "min": 0,
                "max": 15000,
                "mean": 7500,
                "std": 3000
            }
        },
        {
            "name": "height",
            "type": "numeric",
            "values": {
                "min": 1.5,
                "max": 2.1,
                "mean": 1.75,
                "std": 0.1
            }
        },
        {
            "name": "rating",
            "type": "numeric",
            "values": {
                "min": 1,
                "max": 5,
                "mean": 3,
                "std": 1
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

In [832]:
conf_list = [d3, d2, d1, d4, d5, d6]

dataframe_list = build_dataframes(conf_list)

In [836]:
# En caso que se requiera hacer una limpieza de datos y se quiera redondear a dos decimales la altura.
# dataframe_list[5]['height'] = np.round(dataframe_list[5]['height'], 2)
dataframe_list[5]

,id,department,location,status,salary,experience_years,bonus,height,rating
0,4fbf7755075a4f40,Sales,Austin,Contract,49076.604,30.738,3719.043,1.61,3.002
1,28b3c5464f434f9c,Engineering,New York,Full-time,42947.892,26.886,6227.249,1.80,3.022
2,ae28245aa1f344dc,Administration,Austin,Contract,64049.781,34.492,12009.905,1.70,3.496
3,56d90a269c94442d,Administration,San Francisco,Contract,109715.422,34.213,9120.378,1.63,3.629
4,8371c16036ab476e,Support,New York,Part-time,77114.472,10.388,8657.614,1.58,3.178
...,...,...,...,...,...,...,...,...,...
995,2778f36bfd924f48,Sales,Austin,Full-time,103691.332,5.015,5739.101,1.72,4.866
996,4f77bd85ec1e4396,Administration,San Francisco,Full-time,115870.774,16.601,7652.903,1.83,2.579
997,9b79910382f541b6,Engineering,Seattle,Full-time,83021.573,6.414,7494.088,1.68,2.913
998,fb1893eb147249cb,Support,New York,Contract,103035.202,0.729,7468.522,1.67,4.187


## Requisito #5

In [839]:
# dataframe_list[4] en este caso es el dataframe con random = True
simulation_extended = dataframe_list[5]

In [841]:
simulation_extended

,id,department,location,status,salary,experience_years,bonus,height,rating
0,4fbf7755075a4f40,Sales,Austin,Contract,49076.604,30.738,3719.043,1.61,3.002
1,28b3c5464f434f9c,Engineering,New York,Full-time,42947.892,26.886,6227.249,1.80,3.022
2,ae28245aa1f344dc,Administration,Austin,Contract,64049.781,34.492,12009.905,1.70,3.496
3,56d90a269c94442d,Administration,San Francisco,Contract,109715.422,34.213,9120.378,1.63,3.629
4,8371c16036ab476e,Support,New York,Part-time,77114.472,10.388,8657.614,1.58,3.178
...,...,...,...,...,...,...,...,...,...
995,2778f36bfd924f48,Sales,Austin,Full-time,103691.332,5.015,5739.101,1.72,4.866
996,4f77bd85ec1e4396,Administration,San Francisco,Full-time,115870.774,16.601,7652.903,1.83,2.579
997,9b79910382f541b6,Engineering,Seattle,Full-time,83021.573,6.414,7494.088,1.68,2.913
998,fb1893eb147249cb,Support,New York,Contract,103035.202,0.729,7468.522,1.67,4.187


In [843]:
# Arreglo de columnas categoricas como numericas del dataframe al cual se le esta realizando el analisis

# category_cols = ['area', 'subarea']
# numeric_cols = ['income', 'goal']

category_cols = ['department', 'location', 'status']
numeric_cols = ['salary', 'experience_years', 'bonus', 'height', 'rating']

## Analisis mejor combinacion de variables categoricas 

In [846]:
simulation_extended['n'] = 1
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False
).agg({
    'n': 'count'
})

grouped_combination.columns =  category_cols + ['count']
grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[5].shape[0]

El resultado de este analisis se imprime mas abajo para realizar la comparacion con lo que pasa despues de generar los n cantidad de registros aleatorios

In [849]:
# area_subarea sera el resultado que devuelve la funcion get_categoricas_dataset_simulated
final_simulation = get_categorical_dataset_simulated(simulation_extended, category_cols, 100000)
final_simulation

,department,location,status
24,Engineering,San Francisco,Contract
6,Administration,New York,Intern
28,Engineering,Seattle,Contract
44,Sales,Seattle,Contract
51,Support,Austin,Part-time
...,...,...,...
20,Engineering,New York,Contract
38,Sales,New York,Intern
37,Sales,New York,Full-time
10,Administration,San Francisco,Intern


## Analisis mejor combinacion de variables categoricas despues de generar los n cantidad de registros

In [852]:
final_simulation['n'] = 1
result = final_simulation.groupby(
    category_cols,
    as_index=False,
).agg({
    'n': 'count'
})

result.columns = category_cols + ['count']
result['prob'] = result['count'] / final_simulation.shape[0]

## Comparacion del analisis antes y despues de generar los n cantidad de registros

In [855]:
# Mejor combinacion de variables categoricas desde la primera agrupacion
grouped_combination.sort_values(by='count', ascending=False)

,department,location,status,count,prob
0,Administration,Austin,Contract,22,0.022
6,Administration,New York,Intern,22,0.022
58,Support,San Francisco,Intern,21,0.021
43,Sales,San Francisco,Part-time,21,0.021
42,Sales,San Francisco,Intern,21,0.021
...,...,...,...,...,...
36,Sales,New York,Contract,9,0.009
59,Support,San Francisco,Part-time,9,0.009
49,Support,Austin,Full-time,9,0.009
19,Engineering,Austin,Part-time,9,0.009


In [857]:
# Probabilidades de las mejores combinaciones despues de haber generado los 100k registros
result.sort_values(by='count', ascending=False)

,department,location,status,count,prob
0,Administration,Austin,Contract,2216,0.02216
6,Administration,New York,Intern,2215,0.02215
58,Support,San Francisco,Intern,2138,0.02138
14,Administration,Seattle,Intern,2132,0.02132
27,Engineering,San Francisco,Part-time,2112,0.02112
...,...,...,...,...,...
19,Engineering,Austin,Part-time,906,0.00906
36,Sales,New York,Contract,875,0.00875
35,Sales,Austin,Part-time,872,0.00872
49,Support,Austin,Full-time,845,0.00845


Como se puede ver en el ejemplo anterior, las probabilidades despues de haber generado los 100k registros se mantiene "casi" igual como desde la agrupacion original.

In [860]:
final_simulation

,department,location,status,n
24,Engineering,San Francisco,Contract,1
6,Administration,New York,Intern,1
28,Engineering,Seattle,Contract,1
44,Sales,Seattle,Contract,1
51,Support,Austin,Part-time,1
...,...,...,...,...
20,Engineering,New York,Contract,1
38,Sales,New York,Intern,1
37,Sales,New York,Full-time,1
10,Administration,San Francisco,Intern,1


In [862]:
# simulated en este caso es una copia de final_simulation excluyendo la variable n que no pertenece al dataframe original
# final_simulation contiene los 100k registros generados aleatoriamente
simulated = final_simulation.drop(columns='n', axis=1).copy()

# Restablece los indices a una secuencia por default antes: [0, 0, 0, ..., n, n] ahora: [0,1,2,3,...,n]
final_simulation = simulated.sort_index().reset_index(drop=True).copy()

# Recorremos el arreglo de columnas numericas
for nc in numeric_cols:
    # Por cada valor numerico, se generan los demas registros aleatoriamente
    # simulated = copia de los 100k registros generados aleatorios
    # simulation_extended = dataframe al cual se le esta realizando el analisis. En este caso este dataframe se creo desde el inicio con 1k registros
    # category_cols = arreglo con las columnas categoricas
    # nc = columna numerica actual a medida se recorre el ciclo de las columnas numericas
    dfn = get_numeric_column_simulated(simulated, simulation_extended, category_cols, nc) 

    # Combina el dataframe actual de final_simulation con el dataframe que se va generando a medida recorra el ciclo
    # Los combina por sus indices (Tomando en cuenta que los indices son unicos)
    final_simulation = pd.merge(
        final_simulation
        , dfn.loc[:,[nc]]
        , left_index=True
        , right_index=True
    )

In [863]:
# En caso que se requiera hacer una limpieza de datos en la altuna y ponerlo a 2 valores decimales
# final_simulation['height'] = np.round(final_simulation['height'], 2)
final_simulation

,department,location,status,salary,experience_years,bonus,height,rating
0,Administration,Austin,Contract,44968.691,12.831,10253.494,1.869,2.904
1,Administration,Austin,Contract,95571.182,17.101,9287.746,1.666,3.015
2,Administration,Austin,Contract,84059.425,19.423,7471.352,1.834,2.534
3,Administration,Austin,Contract,82916.133,31.392,5007.640,1.702,2.748
4,Administration,Austin,Contract,69514.694,11.106,6923.789,1.728,3.338
...,...,...,...,...,...,...,...,...
99995,Support,Seattle,Part-time,59222.310,20.376,4172.747,1.736,2.801
99996,Support,Seattle,Part-time,60129.381,12.572,3677.562,1.849,3.204
99997,Support,Seattle,Part-time,89521.745,28.024,6524.489,1.836,4.160
99998,Support,Seattle,Part-time,69552.990,10.875,7456.539,1.770,2.807
